In the quest for efficiency and effectiveness in urban transportation, finding the optimal routes to take passengers from their initial locations to their desired destinations is paramount. This challenge is not just about reducing travel time; it's about enhancing the overall experience for both drivers and passengers, ensuring safety, and minimizing environmental impact. 

You have been asked to revolutionize the way taxis navigate the urban landscape, ensuring passengers reach their destinations swiftly, safely, and satisfactorily. As an initial step, your goal is to build a reinforcement learning agent that solves this problem within a simulated environment.

## The Taxi-v3 environment
The Taxi-v3 environment is a strategic simulation, offering a grid-based arena where a taxi navigates to address daily challenges akin to those faced by a taxi driver. This environment is defined by a 5x5 grid where the taxi's mission involves picking up a passenger from one of four specific locations (marked as Red, Green, Yellow, and Blue) and dropping them off at another designated spot. The goal is to accomplish this with minimal time on the road to maximize rewards, emphasizing the need for route optimization and efficient decision-making for passenger pickup and dropoff.

### Key Components:
- **Action Space:** Comprises six actions where 0 moves the taxi south, 1 north, 2 east, 3 west, 4 picks up a passenger, and 5 drops off a passenger.
- **Observation Space:** Comprises 500 discrete states, accounting for 25 taxi positions, 5 potential passenger locations, and 4 destinations. 
- **Rewards System:** Includes a penalty of -1 for each step taken without other rewards, +20 for successful passenger delivery, and -10 for illegal pickup or dropoff actions. Actions resulting in no operation, like hitting a wall, also incur a time step penalty.

![Taxi-v3 environment snapshot](Taxi_snap.png)


In [ ]:
import numpy as np
import gymnasium as gym
import imageio
from IPython.display import Image
from gymnasium.utils import seeding

# Initialize the Taxi-v3 environment
env = gym.make("Taxi-v3", render_mode='rgb_array')

# Seed the environment for reproducibility
env.np_random, _ = seeding.np_random(42)
env.action_space.seed(42)
np.random.seed(42)

# Maximum number of actions per training episode
max_actions = 100
num_episodes = 2000
alpha = 0.1
gamma = 1
epsilon = 0.7

num_states, num_actions = env.observation_space.n, env.action_space.n
q_table = np.zeros((num_states, num_actions))

episode_returns = []

In [ ]:
# Function to update Q-table when learning
def update_q_table(state, action, reward, new_state):
    old_value = q_table[state, action]
    next_max = max(q_table[new_state])
    q_table[state, action] = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)

# Function to select whether choose action randomly or not
def epsilon_greedy(state):
    if np.random.rand() < epsilon:
        return env.action_space.sample()
    else:
        return np.argmax(q_table[state,:])

# Training model loop
for episode in range(num_episodes):
    state, info = env.reset()
    terminated = False
    episode_reward = 0
    i = 0
    while not terminated and i < max_actions:
        action = epsilon_greedy(state)
        new_state, reward, terminated, truncated, info = env.step(action)
        update_q_table(state, action, reward, new_state)
        episode_reward += reward
        state = new_state
        i += 1
    episode_returns.append(episode_reward)

print(episode_returns)
print(q_table)

[[  0.           0.           0.           0.           0.
    0.        ]
 [ -7.45049025  -6.43142359  -7.44715431  -6.4281101   -5.38108764
  -15.44699201]
 [ -2.0692502   -0.73718849  -2.58522781  -0.66659936   1.34515624
   -9.64532018]
 ...
 [ -0.38125836   1.7737254   -0.47751953  -1.20967678  -9.50640622
   -9.45458984]
 [ -5.92730484  -4.68262586  -5.92689554  -4.74006417 -14.99160636
  -14.99261174]
 [  5.84852634   3.58195933   5.60560983  13.96245916  -2.02520169
   -1.75888895]]


In [ ]:
# Save the best policy
policy = {state:np.argmax(q_table[state]) for state in range(num_states)}
print(policy)

In [ ]:
# Test the environment
env.reset(seed=42)
done = False
episode_total_reward = 0
x = 1
frames = []

while not done:
    action = policy[state]  # optimal action from policy
    frames.append(env.render())
    step_result = env.step(action)
    if len(step_result) == 5:
        next_state, reward, done, truncated, info = step_result
    else:
        next_state, reward, done, info = step_result
    state = next_state
    episode_total_reward += reward
    print(f"Step {x}")
    x += 1

print(f"Total episode reward is: {episode_total_reward}")

In [ ]:
# Once you are done, run this cell to visualize the agent's behavior through the episode
# Save frames as a GIF
imageio.mimsave('taxi_agent_behavior.gif', frames, fps=5, loop=0)

# Display GIF
gif_path = "taxi_agent_behavior.gif" 
Image(gif_path) 